# Fractional Brownian motion application

## CPU info

In [1]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: AuthenticAMD
cpu family	: 25
model		: 17
model name	: AMD EPYC 9B14
stepping	: 1
microcode	: 0xffffffff
cpu MHz		: 2599.998
cache size	: 1024 KB
physical id	: 0
siblings	: 8
core id		: 0
cpu cores	: 4
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 16
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a misalignsse 3dnowprefetch osvw topoext invpcid_single ssbd ibrs ibpb stibp vmmcall fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid avx512f avx512dq rdseed adx smap avx512ifma clflushopt clwb avx512cd sha_ni avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves avx512_bf16 clzero xsaveerptr wbnoinvd arat avx512v

Install relevant packages which are not installed yet

In [3]:
!pip install PyGithub
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 3.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 17.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 48.4 MB/s eta 0:00:0000:0100:01


## Run for N_T=9

One of the test runs

In [5]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
auth = Auth.Token('*******') # Authentication has been removed for security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from fBMABLS import main as AndersonBroadie2004
from fBMHK import main as Glassermann
from fBMBelomestny import main as mainBelomestnyetal2009
from fBMSAA import main as SAA_LP
from fBMSAABelomestny import main as BelomestnySAA2013
from fBMSchoenmakersSZH import main as SZH2013

import numpy as np
import pandas as pd
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 40000 
traj_est_primal_dual = 700000
traj_est_MM = 8000
traj_test_ub_desai2=30000
traj_est_MM_belomestny2013=15000
traj_lb_testing = 100000
information=[]
steps=9


T=1
basis_f_K_U_MM=80
basis_f_K_U_MM_desai2=70
basis_f_K_U_MM_belomestny2013=200
basis_f_K_U_MM_BHS=int(basis_f_K_U_MM*1.2)

K_L_basic= 250
K_U_belomestny2009= 200
K_U_SZH2013=160
K_L_AndersenBroadie2004=250
K_L_Glasserman2004= 250
grid_Glasserman2004= 2000
inner_sim_SZH_=1
inner_sim_AB= 1000 
fine_grid_belomestny2009_= 1

traj_test_ub_AB=traj_test_ub//2
traj_lb_testing_GM=traj_test_ub//2

inner_sim=600
i=0
if __name__=='__main__':
    for d,H in  [(1,0.2)]:
        for basis_f_K_U_MM_belomestny2013, traj_est_MM_belomestny2013 in [(210,15000), (90, 50000), (120, 35000)]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))     
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols = [d, H, '', '', '', '']


#             label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_, d,  ''.join(np.repeat('-', 10)))
#             list_inf=SZH2013(d, True, hurst=H, steps=steps,T=T, mode_kaggle=True, grid=1, K_low=K_L_basic,K_up=K_U_SZH2013, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9)
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_, info_cols=inf_cols)
#             information.append(inf_list)    

#             label_=f'Belom. et al. (2009) KU100-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_, d,  ''.join(np.repeat('-', 10)))
#             list_inf=mainBelomestnyetal2009(d, True,hurst=H, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_, K_low=K_L_basic,K_up=K_U_belomestny2009, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9) 
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_, info_cols=inf_cols)
#             information.append(inf_list)   


#             K_L_AB=K_L_AndersenBroadie2004
#             K_L_GM=K_L_Glasserman2004
#             label_ = f'Glasserman (2004)-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004, d, ''.join(np.repeat('-', 10)))
#             list_inf=Glassermann(d, True, hurst=H, steps=steps,T=T, grid=grid_Glasserman2004, K_low=K_L_GM, traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, seed=i+9) 
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004, info_cols=inf_cols)
#             information.append(inf_list)   

#             label_ = f'Anderson Broadie (2004)-{steps}'
#             print(''.join(np.repeat('-', 10)), label_, inner_sim_AB, d, ''.join(np.repeat('-', 10)))
#             list_inf=AndersonBroadie2004(d, True, hurst=H, steps=steps,T=T, grid= inner_sim_AB, K_low=K_L_AB, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, seed=i+9) 
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB, info_cols=inf_cols)
#             information.append(inf_list)


#             label_ = f'Belomestny et al. (2019)-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
#             list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS,  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9,  mode_desai_BBS_BHS='bhs')
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
#             information.append(inf_list)  

#             label_ = f'Belomestny et al. (2023) best-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
#             list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
#             information.append(inf_list)  

#             label_ = f'Belomestny et al. (2023) original-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
#             list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
#             information.append(inf_list)  

#             label_ =f'Desai et al. (2012)-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
#             list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
#             information.append(inf_list)  

#             label_ =f'Desai et al. (2012) high test-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
#             list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=int(traj_est_MM*1.5),traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
#             information.append(inf_list)  

            for lambda_ in [1]:
                label_ = f'Belomesty (2013); lambda={lambda_}'
                print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
                if lambda_==1/20:
                    traj_est_MM_belomestny2013_=int(1.1*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.1*basis_f_K_U_MM_belomestny2013)
                elif lambda_==1/2:
                    traj_est_MM_belomestny2013_=int(1.05*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.05*basis_f_K_U_MM_belomestny2013)
                else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                list_inf=BelomestnySAA2013(d, True, hurst=H, steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, lambda_=lambda_, p=100)
                inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                information.append(inf_list) 





            # with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
            #     pic.dump(information, fh)
            # repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    info_original=pd.DataFrame.from_records(information)
    info_transformed=pd.DataFrame.from_records(utils.information_format_fbm_no_fu(information), columns=utils.header_fbm_no_fu)
    lower_bounds_cv=info_original.iloc[:,[2,13, 14]]
    upper_bounds = info_original.iloc[:, 9]
    loc_lowest_standard_error = lower_bounds_cv.groupby(2)[14].idxmin()
    lower_bounds_cv_sel = lower_bounds_cv.loc[loc_lowest_standard_error, [2, 13]].rename(columns={13: 'best'})
    lower_bounds_cv=pd.merge(left= lower_bounds_cv, right=lower_bounds_cv_sel, on=2)['best']
    lower_bounds_cv.index= info_original.index
    info_transformed['GAP']= upper_bounds- lower_bounds_cv
    table_ = tabulate(utils.information_format_fbm_no_fu(information), headers=utils.header_fbm_no_fu, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)

********** 600 **********
---------- Belomesty (2013); lambda=1 600 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 1.4120841405487856   - Norm Jacobian 13.932055867669954 - Max abs Jacobian 0.6483617370760778
Iteration: 10, Objective Value: 0.857004859385253   - Norm Jacobian 9.657776621858718 - Max abs Jacobian 0.42408834449834293
Iteration: 20, Objective Value: 0.5074557029388068   - Norm Jacobian 6.793803065437518 - Max abs Jacobian 0.3721374923318303
Iteration: 30, Objective Value: 0.4299209148421078   - Norm Jacobian 2.170240423002521 - Max abs Jacobian 0.1563033119695707
Iteration: 40, Objective Value: 0.4121371758047905   - Norm Jacobian 0.7565120324313321 - Max abs Jacobian 0.04680284763042589
Iteration: 50, Objective Value: 0.4087624097385792   - Norm Jacobian 0.3986783256443105 - Max abs Jacobian 0.025993623933302096
Iteration: 60, Objective Value: 0.40745032698622363   - Norm Jacobian 0.22310040972142794 - Max abs Jacobian 0.013997265043807185
Iteration: 70, Objective Value: 0.4070110107938792   - Norm Jacobian 0.09235641346325622 - Max ab

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.9327959333535079   - Norm Jacobian 8.498684471617148 - Max abs Jacobian 0.6660129093771446
Iteration: 10, Objective Value: 0.5500298663921701   - Norm Jacobian 3.4789062223787943 - Max abs Jacobian 0.3185576940164003
Iteration: 20, Objective Value: 0.43933662820199476   - Norm Jacobian 1.3434627831585417 - Max abs Jacobian 0.17970360068309787
Iteration: 30, Objective Value: 0.42420118589838146   - Norm Jacobian 0.3433203252431574 - Max abs Jacobian 0.06333817814813601
Iteration: 40, Objective Value: 0.42108933573668705   - Norm Jacobian 0.1220481638904471 - Max abs Jacobian 0.016405074655222306
Iteration: 50, Objective Value: 0.4204171806313843   - Norm Jacobian 0.07025348834372123 - Max abs Jacobian 0.0065544927963816875
Iteration: 60, Objective Value: 0.42028440554511953   - Norm Jacobian 0.02863739252696558 - Max abs Jacobian 0.0033101977596815208
Iteration: 70, Objective Value: 0.42025142592311726   - Norm Jacobian 0.01697089094339

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 1.0635075231329467   - Norm Jacobian 10.075472683050743 - Max abs Jacobian 0.6735143824857145
Iteration: 10, Objective Value: 0.6537743516596002   - Norm Jacobian 7.281160845217853 - Max abs Jacobian 0.36807814750215484
Iteration: 20, Objective Value: 0.4575453072103576   - Norm Jacobian 2.9544503276179865 - Max abs Jacobian 0.2740164985019414
Iteration: 30, Objective Value: 0.4226222829069896   - Norm Jacobian 1.0332541604647323 - Max abs Jacobian 0.09694723487048743
Iteration: 40, Objective Value: 0.41689433954981175   - Norm Jacobian 0.25026048790252814 - Max abs Jacobian 0.031804162875657106
Iteration: 50, Objective Value: 0.4155018929018032   - Norm Jacobian 0.12419792425866101 - Max abs Jacobian 0.009544908405729618
Iteration: 60, Objective Value: 0.4151579110806616   - Norm Jacobian 0.06120950167612612 - Max abs Jacobian 0.005446451924521957
Iteration: 70, Objective Value: 0.41506583884306847   - Norm Jacobian 0.03438907347761629 

### Final run (for $H\in\{0.2,0.3, 0.45, ,0.7\}$)

In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
auth = Auth.Token('*******') # Authentication has been removed for security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from fBMABLS import main as AndersonBroadie2004
from fBMHK import main as Glassermann
from fBMBelomestny import main as mainBelomestnyetal2009
from fBMSAA import main as SAA_LP
from fBMSAABelomestny import main as BelomestnySAA2013
from fBMSchoenmakersSZH import main as SZH2013

import numpy as np
import pandas as pd
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 700000
traj_est_MM = 8000
traj_test_ub_desai2=30000
traj_est_MM_belomestny2013=15000
traj_lb_testing = 100000
information=[]
steps=9


T=1
basis_f_K_U_MM=80
basis_f_K_U_MM_desai2=50
basis_f_K_U_MM_belomestny2013=230
basis_f_K_U_MM_BHS=int(basis_f_K_U_MM*1.15)

K_L_basic= 250
K_U_belomestny2009= 200
K_U_SZH2013=160
K_L_AndersenBroadie2004=250
K_L_Glasserman2004= 250
grid_Glasserman2004= 2400
inner_sim_SZH_=1
inner_sim_AB= 1000 
fine_grid_belomestny2009_= 1

traj_test_ub_AB=traj_test_ub//2
traj_lb_testing_GM=traj_test_ub//2


i=0
if __name__=='__main__':
    for d,H in  [(1,0.2), (1,0.3), (1, 0.45), (1, 0.7)]:
        for inner_sim in [600]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))     
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols = [d, H, '', '', '', '']


            label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_, d,  ''.join(np.repeat('-', 10)))
            list_inf=SZH2013(d, True, hurst=H, steps=steps,T=T, mode_kaggle=True, grid=1, K_low=K_L_basic,K_up=K_U_SZH2013, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_, info_cols=inf_cols)
            information.append(inf_list)    

            label_=f'Belom. et al. (2009) KU100-{steps}'
            print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_, d,  ''.join(np.repeat('-', 10)))
            list_inf=mainBelomestnyetal2009(d, True,hurst=H, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_, K_low=K_L_basic,K_up=K_U_belomestny2009, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_, info_cols=inf_cols)
            information.append(inf_list)   


            K_L_AB=K_L_AndersenBroadie2004
            K_L_GM=K_L_Glasserman2004
            label_ = f'Glasserman (2004)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004, d, ''.join(np.repeat('-', 10)))
            list_inf=Glassermann(d, True, hurst=H, steps=steps,T=T, grid=grid_Glasserman2004, K_low=K_L_GM, traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004, info_cols=inf_cols)
            information.append(inf_list)   

            label_ = f'Anderson Broadie (2004)-{steps}'
            print(''.join(np.repeat('-', 10)), label_, inner_sim_AB, d, ''.join(np.repeat('-', 10)))
            list_inf=AndersonBroadie2004(d, True, hurst=H, steps=steps,T=T, grid= inner_sim_AB, K_low=K_L_AB, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB, info_cols=inf_cols)
            information.append(inf_list)


            label_ = f'Belomestny et al. (2019)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS,  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9,  mode_desai_BBS_BHS='bhs')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) best-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) original-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012) high test-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=int(traj_est_MM*1.5),traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            for lambda_ in [1/20, 1/2, 1]:
                label_ = f'Belomesty (2013); lambda={lambda_}'
                print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
                if lambda_==1/20:
                    traj_est_MM_belomestny2013_=int(1.1*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.1*basis_f_K_U_MM_belomestny2013)
                elif lambda_==1/2:
                    traj_est_MM_belomestny2013_=int(1.05*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.05*basis_f_K_U_MM_belomestny2013)
                else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                list_inf=BelomestnySAA2013(d, True, hurst=H, steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, lambda_=lambda_, p=100)
                inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                information.append(inf_list) 





            with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
                pic.dump(information, fh)
            repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    info_original=pd.DataFrame.from_records(information)
    info_transformed=pd.DataFrame.from_records(utils.information_format_fbm_no_fu(information), columns=utils.header_fbm_no_fu)
    lower_bounds_cv=info_original.iloc[:,[2,13, 14]]
    upper_bounds = info_original.iloc[:, 9]
    loc_lowest_standard_error = lower_bounds_cv.groupby(2)[14].idxmin()
    lower_bounds_cv_sel = lower_bounds_cv.loc[loc_lowest_standard_error, [2, 13]].rename(columns={13: 'best'})
    lower_bounds_cv=pd.merge(left= lower_bounds_cv, right=lower_bounds_cv_sel, on=2)['best']
    lower_bounds_cv.index= info_original.index
    info_transformed['GAP']= upper_bounds- lower_bounds_cv
    table_ = tabulate(utils.information_format_fbm_no_fu(information), headers=utils.header_fbm_no_fu, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)


********** 600 **********
---------- Schoenmakers et al. (2013) KU100 -9 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


8
7
6
5
4
3
2
1
0
0
1
2
3
4
5
6
7
8
Lowerbound
Value 0.3610490448286773
Std 0.0026944690644603547
Stop time-mean 5.11545
Stop time-std 2.5928751025647183

Upperbound
up 0.38474580803729636
std 0.0012841681882568075
up2 0.3746572508372763
std2 0.0014323533365769312
CV est 0.3612960002906745
CV std 0.0014005187189053791
time avg training 0:02:14.326467
time avg ub 0:00:00.059664
---------- Belom. et al. (2009) KU100-9 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
0
1
2
3
4
5
6
7
8
[0.00813707 0.00940951 0.00771455]
Lowerbound
Value 0.36012947776892434
Std 0.002701927171374924
Stop time-mean 5.14288
Stop time-std 2.5826547011941026
Upperbound
up 0.38489045564328617
std 0.001298647961929795
up2 0.37412934163220646
std2 0.0014488105404998706
CV est 0.3605423335944729
CV std 0.0014074798064049251
time avg training 0:01:31.036513
time avg ub 0:00:00.044396
---------- Glasserman (2004)-9 2400 1 ----------


...

8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.19111352375401525   - Norm Jacobian 2.946796726274097 - Max abs Jacobian 0.11680362401827572
Iteration: 10, Objective Value: 0.16957096816583972   - Norm Jacobian 1.022475048824272 - Max abs Jacobian 0.04519961553271263
Iteration: 20, Objective Value: 0.16556738848421304   - Norm Jacobian 0.17943280868890465 - Max abs Jacobian 0.012334073672472412
Iteration: 30, Objective Value: 0.16360896642026607   - Norm Jacobian 0.12696663638436337 - Max abs Jacobian 0.008540707579184894
Iteration: 40, Objective Value: 0.16266461678284003   - Norm Jacobian 0.07214089358148161 - Max abs Jacobian 0.004581321148398969
Iteration: 50, Objective Value: 0.16239989239193728   - Norm Jacobian 0.01797491046860102 - Max abs Jacobian 0.0016302621710826658
Iteration: 60, Objective Value: 0.1623463242622934   - Norm Jacobian 0.006800497577473615 - Max abs Jacobian 0.0007432623136780872
Iteration: 70, Objective Value: 0.1623378574495354   - Norm Jacobian 0.004263

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.2493426320276623   - Norm Jacobian 6.37283628493215 - Max abs Jacobian 0.23129630280622274
Iteration: 10, Objective Value: 0.1923655310005436   - Norm Jacobian 4.789839235334749 - Max abs Jacobian 0.2384879181331636
Iteration: 20, Objective Value: 0.18223599794598155   - Norm Jacobian 0.6972551249261204 - Max abs Jacobian 0.04922276572943523
Iteration: 30, Objective Value: 0.17954328173910697   - Norm Jacobian 0.28449851821470806 - Max abs Jacobian 0.01630992518709791
Iteration: 40, Objective Value: 0.17891053340726654   - Norm Jacobian 0.17749173698999096 - Max abs Jacobian 0.010595617858054431
Iteration: 50, Objective Value: 0.1786492180332293   - Norm Jacobian 0.04366298126170164 - Max abs Jacobian 0.0038626256743736656
Iteration: 60, Objective Value: 0.1785550430536636   - Norm Jacobian 0.02818085867988624 - Max abs Jacobian 0.0016716738093566776
Iteration: 70, Objective Value: 0.1785268927340315   - Norm Jacobian 0.013153104037120

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.31004743796534195   - Norm Jacobian 10.542927644745951 - Max abs Jacobian 0.5085864763288614
Iteration: 10, Objective Value: 0.2220707423109242   - Norm Jacobian 6.7074449790919335 - Max abs Jacobian 0.31972211434207076
Iteration: 20, Objective Value: 0.20109388910064271   - Norm Jacobian 3.007857063602664 - Max abs Jacobian 0.17528131546664208
Iteration: 30, Objective Value: 0.19524827996393707   - Norm Jacobian 0.6029282878626456 - Max abs Jacobian 0.04742714495255727
Iteration: 40, Objective Value: 0.19370018040904183   - Norm Jacobian 0.2597070977652238 - Max abs Jacobian 0.018153968188927665
Iteration: 50, Objective Value: 0.19319996054096156   - Norm Jacobian 0.13019876388716278 - Max abs Jacobian 0.011521682639963223
Iteration: 60, Objective Value: 0.19300525155923465   - Norm Jacobian 0.10491489784955853 - Max abs Jacobian 0.006902009574859696
Iteration: 70, Objective Value: 0.19292074418150754   - Norm Jacobian 0.0570555799094

### Test run H=0.45

In [1]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
auth = Auth.Token('*******') # Authentication has been removed for security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from fBMABLS import main as AndersonBroadie2004
from fBMHK import main as Glassermann
from fBMBelomestny import main as mainBelomestnyetal2009
from fBMSAA import main as SAA_LP
from fBMSAABelomestny import main as BelomestnySAA2013
from fBMSchoenmakersSZH import main as SZH2013

import numpy as np
import pandas as pd
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 700000
traj_est_MM = 8000
traj_test_ub_desai2=30000
traj_est_MM_belomestny2013=15000
traj_lb_testing = 100000
information=[]
steps=9


T=1
basis_f_K_U_MM=80
basis_f_K_U_MM_desai2=50
basis_f_K_U_MM_belomestny2013=230
basis_f_K_U_MM_BHS=int(basis_f_K_U_MM*1.15)

K_L_basic= 250
K_U_belomestny2009= 200
K_U_SZH2013=160
K_L_AndersenBroadie2004=250
K_L_Glasserman2004= 250
grid_Glasserman2004= 2400
inner_sim_SZH_=1
inner_sim_AB= 1000 
fine_grid_belomestny2009_= 1

traj_test_ub_AB=traj_test_ub//2
traj_lb_testing_GM=traj_test_ub//2


i=0
if __name__=='__main__':
    for d,H in  [(1,0.45)]:
        for inner_sim in [600]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))     
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols = [d, H, '', '', '', '']


            label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_, d,  ''.join(np.repeat('-', 10)))
            list_inf=SZH2013(d, True, hurst=H, steps=steps,T=T, mode_kaggle=True, grid=1, K_low=K_L_basic,K_up=K_U_SZH2013, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_, info_cols=inf_cols)
            information.append(inf_list)    

            label_=f'Belom. et al. (2009) KU100-{steps}'
            print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_, d,  ''.join(np.repeat('-', 10)))
            list_inf=mainBelomestnyetal2009(d, True,hurst=H, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_, K_low=K_L_basic,K_up=K_U_belomestny2009, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_, info_cols=inf_cols)
            information.append(inf_list)   


            K_L_AB=K_L_AndersenBroadie2004
            K_L_GM=K_L_Glasserman2004
            label_ = f'Glasserman (2004)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004, d, ''.join(np.repeat('-', 10)))
            list_inf=Glassermann(d, True, hurst=H, steps=steps,T=T, grid=grid_Glasserman2004, K_low=K_L_GM, traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004, info_cols=inf_cols)
            information.append(inf_list)   

            label_ = f'Anderson Broadie (2004)-{steps}'
            print(''.join(np.repeat('-', 10)), label_, inner_sim_AB, d, ''.join(np.repeat('-', 10)))
            list_inf=AndersonBroadie2004(d, True, hurst=H, steps=steps,T=T, grid= inner_sim_AB, K_low=K_L_AB, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB, info_cols=inf_cols)
            information.append(inf_list)


            label_ = f'Belomestny et al. (2019)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS,  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9,  mode_desai_BBS_BHS='bhs')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) best-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) original-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012) high test-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=int(traj_est_MM*1.5),traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            for lambda_ in [1/20, 1/2, 1]:
                label_ = f'Belomesty (2013); lambda={lambda_}'
                print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
                if lambda_==1/20:
                    traj_est_MM_belomestny2013_=int(1.1*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.1*basis_f_K_U_MM_belomestny2013)
                elif lambda_==1/2:
                    traj_est_MM_belomestny2013_=int(1.05*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.05*basis_f_K_U_MM_belomestny2013)
                else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                list_inf=BelomestnySAA2013(d, True, hurst=H, steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, lambda_=lambda_, p=100)
                inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                information.append(inf_list) 





            with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
                pic.dump(information, fh)
            repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    info_original=pd.DataFrame.from_records(information)
    info_transformed=pd.DataFrame.from_records(utils.information_format_fbm_no_fu(information), columns=utils.header_fbm_no_fu)
    lower_bounds_cv=info_original.iloc[:,[2,13, 14]]
    upper_bounds = info_original.iloc[:, 9]
    loc_lowest_standard_error = lower_bounds_cv.groupby(2)[14].idxmin()
    lower_bounds_cv_sel = lower_bounds_cv.loc[loc_lowest_standard_error, [2, 13]].rename(columns={13: 'best'})
    lower_bounds_cv=pd.merge(left= lower_bounds_cv, right=lower_bounds_cv_sel, on=2)['best']
    lower_bounds_cv.index= info_original.index
    info_transformed['GAP']= upper_bounds- lower_bounds_cv
    table_ = tabulate(utils.information_format_fbm_no_fu(information), headers=utils.header_fbm_no_fu, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)


2024-06-14 04:04:19.526218: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-14 04:04:20.112297: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 04:04:20.112367: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 04:04:20.221048: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-14 04:04:20.440725: I tensorflow/core/platform/cpu_feature_guar

********** 600 **********
---------- Schoenmakers et al. (2013) KU100 -9 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


8
7
6
5
4
3
2
1
0
0
1
2
3
4
5
6
7
8
Lowerbound
Value 0.046557356384548575
Std 0.002421147552165194
Stop time-mean 5.04739
Stop time-std 2.7905885020726364

Upperbound
up 0.0509924862441093
std 0.0001691981742469217
up2 0.04994168954067933
std2 0.00018418706543933943
CV est 0.04834086368253249
CV std 0.00018052730638793026
time avg training 0:02:13.487481
time avg ub 0:00:00.056279
---------- Belom. et al. (2009) KU100-9 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


...

8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.09583424213957979   - Norm Jacobian 5.269456990125371 - Max abs Jacobian 0.3046665305944043
Iteration: 10, Objective Value: 0.06577198100375925   - Norm Jacobian 2.418212665395165 - Max abs Jacobian 0.13100429867302305
Iteration: 20, Objective Value: 0.061630447137810615   - Norm Jacobian 0.810990909127443 - Max abs Jacobian 0.04072805076181178
Iteration: 30, Objective Value: 0.06000685645567697   - Norm Jacobian 0.18568565083371574 - Max abs Jacobian 0.010374111323936544
Iteration: 40, Objective Value: 0.05946669345968252   - Norm Jacobian 0.12840178239482014 - Max abs Jacobian 0.009169713414858586
Iteration: 50, Objective Value: 0.059235195597249206   - Norm Jacobian 0.05139839342931694 - Max abs Jacobian 0.0039056837362550292
Iteration: 60, Objective Value: 0.05914962325017007   - Norm Jacobian 0.02527238433559301 - Max abs Jacobian 0.0018219800667666347
Iteration: 70, Objective Value: 0.059112689940938525   - Norm Jacobian 0.012171

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.1746399442552959   - Norm Jacobian 10.330545954121707 - Max abs Jacobian 0.4585976325033504
Iteration: 10, Objective Value: 0.0970979689676282   - Norm Jacobian 5.878846067884982 - Max abs Jacobian 0.24834919103636205
Iteration: 20, Objective Value: 0.07675425895628474   - Norm Jacobian 2.5358476338009277 - Max abs Jacobian 0.11450557997735664
Iteration: 30, Objective Value: 0.07282455813493138   - Norm Jacobian 0.6084562611453042 - Max abs Jacobian 0.033795083947584424
Iteration: 40, Objective Value: 0.07197646896393878   - Norm Jacobian 0.1984666816838897 - Max abs Jacobian 0.013687607355179482
Iteration: 50, Objective Value: 0.07164222278524077   - Norm Jacobian 0.1743733833420126 - Max abs Jacobian 0.00944305541121045
Iteration: 60, Objective Value: 0.07149280190194653   - Norm Jacobian 0.09292122178595413 - Max abs Jacobian 0.005500023390517832
Iteration: 70, Objective Value: 0.07142733943585548   - Norm Jacobian 0.051076113061671

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.2650489734785705   - Norm Jacobian 14.827873764414502 - Max abs Jacobian 0.6411071121127956
Iteration: 10, Objective Value: 0.13616433866826697   - Norm Jacobian 7.032166800634975 - Max abs Jacobian 0.33820713917747525
Iteration: 20, Objective Value: 0.1010229239108528   - Norm Jacobian 5.46508462507611 - Max abs Jacobian 0.24521507769497297
Iteration: 30, Objective Value: 0.0884278584209536   - Norm Jacobian 1.251916207300181 - Max abs Jacobian 0.0606914083502872
Iteration: 40, Objective Value: 0.08624714515996033   - Norm Jacobian 0.28823912700431137 - Max abs Jacobian 0.022853925324829123
Iteration: 50, Objective Value: 0.08568221338501784   - Norm Jacobian 0.25085181211757496 - Max abs Jacobian 0.01373076848368472
Iteration: 60, Objective Value: 0.08543190989374884   - Norm Jacobian 0.1279089254905942 - Max abs Jacobian 0.007674923621827091
Iteration: 70, Objective Value: 0.08531614633573216   - Norm Jacobian 0.08032229619081194 - 

## N_T=49

Test run

In [1]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
auth = Auth.Token('*******') # Authentication has been removed for security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from fBMABLS import main as AndersonBroadie2004
from fBMHK import main as Glassermann
from fBMBelomestny import main as mainBelomestnyetal2009
from fBMSAA import main as SAA_LP
from fBMSAABelomestny import main as BelomestnySAA2013
from fBMSchoenmakersSZH import main as SZH2013

import numpy as np
import pandas as pd
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 400000
traj_est_MM = 2000
traj_test_ub_desai2=30000
traj_est_MM_belomestny2013=6500
traj_lb_testing = 100000
information=[]
steps=49


T=1
basis_f_K_U_MM=80
basis_f_K_U_MM_desai2=50
basis_f_K_U_MM_belomestny2013=230
basis_f_K_U_MM_BHS=int(basis_f_K_U_MM*1.15)

K_L_basic= 100
K_U_belomestny2009= 80
K_U_SZH2013=80
K_L_AndersenBroadie2004=20
K_L_Glasserman2004= 60
grid_Glasserman2004= 1000
inner_sim_SZH_=1
inner_sim_AB= 250 
fine_grid_belomestny2009_= 1

traj_test_ub_AB=traj_test_ub//7
traj_lb_testing_GM=traj_test_ub//2


i=0
if __name__=='__main__':
    for d,H in  [(1,0.2)]:
        for inner_sim in [600]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))     
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols = [d, H, '', '', '', '']


            label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_, d,  ''.join(np.repeat('-', 10)))
            list_inf=SZH2013(d, True, hurst=H, steps=steps,T=T, mode_kaggle=True, grid=1, K_low=K_L_basic,K_up=K_U_SZH2013, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_, info_cols=inf_cols)
            information.append(inf_list)    

            label_=f'Belom. et al. (2009) KU100-{steps}'
            print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_, d,  ''.join(np.repeat('-', 10)))
            list_inf=mainBelomestnyetal2009(d, True,hurst=H, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_, K_low=K_L_basic,K_up=K_U_belomestny2009, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_, info_cols=inf_cols)
            information.append(inf_list)   


            K_L_AB=K_L_AndersenBroadie2004
            K_L_GM=K_L_Glasserman2004
            label_ = f'Glasserman (2004)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004, d, ''.join(np.repeat('-', 10)))
            list_inf=Glassermann(d, True, hurst=H, steps=steps,T=T, grid=grid_Glasserman2004, K_low=K_L_GM, traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004, info_cols=inf_cols)
            information.append(inf_list)   

            label_ = f'Anderson Broadie (2004)-{steps}'
            print(''.join(np.repeat('-', 10)), label_, inner_sim_AB, d, ''.join(np.repeat('-', 10)))
            list_inf=AndersonBroadie2004(d, True, hurst=H, steps=steps,T=T, grid= inner_sim_AB, K_low=K_L_AB, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB, info_cols=inf_cols)
            information.append(inf_list)


            label_ = f'Belomestny et al. (2019)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS,  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9,  mode_desai_BBS_BHS='bhs')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) best-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) original-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012) high test-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=int(traj_est_MM*1.5),traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            for lambda_ in [1/20, 1/2, 1]:
                label_ = f'Belomesty (2013); lambda={lambda_}'
                print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
                if lambda_==1/20:
                    traj_est_MM_belomestny2013_=int(1.1*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.1*basis_f_K_U_MM_belomestny2013)
                elif lambda_==1/2:
                    traj_est_MM_belomestny2013_=int(1.05*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.05*basis_f_K_U_MM_belomestny2013)
                else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                list_inf=BelomestnySAA2013(d, True, hurst=H, steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, lambda_=lambda_, p=100)
                inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                information.append(inf_list) 





            with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
                pic.dump(information, fh)
            repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    info_original=pd.DataFrame.from_records(information)
    info_transformed=pd.DataFrame.from_records(utils.information_format_fbm_no_fu(information), columns=utils.header_fbm_no_fu)
    lower_bounds_cv=info_original.iloc[:,[2,13, 14]]
    upper_bounds = info_original.iloc[:, 9]
    loc_lowest_standard_error = lower_bounds_cv.groupby(2)[14].idxmin()
    lower_bounds_cv_sel = lower_bounds_cv.loc[loc_lowest_standard_error, [2, 13]].rename(columns={13: 'best'})
    lower_bounds_cv=pd.merge(left= lower_bounds_cv, right=lower_bounds_cv_sel, on=2)['best']
    lower_bounds_cv.index= info_original.index
    info_transformed['GAP']= upper_bounds- lower_bounds_cv
    table_ = tabulate(utils.information_format_fbm_no_fu(information), headers=utils.header_fbm_no_fu, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)



2024-06-12 09:46:43.060018: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-12 09:46:43.692752: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 09:46:43.692821: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 09:46:43.802067: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-12 09:46:44.010270: I tensorflow/core/platform/cpu_feature_guar

********** 600 **********
---------- Schoenmakers et al. (2013) KU100 -49 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
Lowerbound
Value 0.5827776886194472
Std 0.002574428468022544
Stop time-mean 25.1606
Stop time-std 13.578917027509963

Upperbound
up 0.6351839880292232
std 0.0013317372570929486
up2 0.6189888495800687
std2 0.0015817665372064172
CV est 0.5819820579514666
CV std 0.0014793176004687483
time avg training 0:03:07.165269
time avg ub 0:00:00.293387
---------- Belom. et al. (2009) KU100-49 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


...

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2418399
Academic license 2418399 - for non-commercial use only - registered to p.___@tilburguniversity.edu
Set parameter BarHomogeneous to value 0
Set parameter OptimalityTol to value 0.0001
Set parameter BarConvTol to value 0.0001
Set parameter Crossover to value 0
Set parameter Method to value 2
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Debian GNU/Linux 11 (bullseye)")

CPU model: AMD EPYC 9B14, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2418399 - for non-commercial use only - registered to p.___@tilburguniversity.edu
Optimize a model with 105840 rows, 9840 columns and 196108914 nonzeros
Model fingerprint: 0x76c91762
Coefficient statistics:
  Matrix range     [

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48



KeyboardInterrupt



Test run

In [2]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
auth = Auth.Token('*******') # Authentication has been removed for security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from fBMABLS import main as AndersonBroadie2004
from fBMHK import main as Glassermann
from fBMBelomestny import main as mainBelomestnyetal2009
from fBMSAA import main as SAA_LP
from fBMSAABelomestny import main as BelomestnySAA2013
from fBMSchoenmakersSZH import main as SZH2013

import numpy as np
import pandas as pd
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 400000
traj_est_MM = 2000
traj_test_ub_desai2=30000
traj_est_MM_belomestny2013=6500
traj_lb_testing = 100000
information=[]
steps=49


T=1
basis_f_K_U_MM=40
basis_f_K_U_MM_desai2=35
basis_f_K_U_MM_belomestny2013=80
basis_f_K_U_MM_BHS=int(basis_f_K_U_MM*1.05)

K_L_basic= 100
K_U_belomestny2009= 80
K_U_SZH2013=80
K_L_AndersenBroadie2004=90
K_L_Glasserman2004= 120
grid_Glasserman2004= 1000
inner_sim_SZH_=1
inner_sim_AB= 500 
fine_grid_belomestny2009_= 1

traj_test_ub_AB=traj_test_ub//7
traj_lb_testing_GM=traj_test_ub//4


i=0
if __name__=='__main__':
    for d,H in  [(1,0.2)]:
        for inner_sim in [600]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))     
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols = [d, H, '', '', '', '']


            label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_, d,  ''.join(np.repeat('-', 10)))
            list_inf=SZH2013(d, True, hurst=H, steps=steps,T=T, mode_kaggle=True, grid=1, K_low=K_L_basic,K_up=K_U_SZH2013, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_, info_cols=inf_cols)
            information.append(inf_list)    

            label_=f'Belom. et al. (2009) KU100-{steps}'
            print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_, d,  ''.join(np.repeat('-', 10)))
            list_inf=mainBelomestnyetal2009(d, True,hurst=H, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_, K_low=K_L_basic,K_up=K_U_belomestny2009, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_, info_cols=inf_cols)
            information.append(inf_list)   


            K_L_AB=K_L_AndersenBroadie2004
            K_L_GM=K_L_Glasserman2004
            label_ = f'Glasserman (2004)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004, d, ''.join(np.repeat('-', 10)))
            list_inf=Glassermann(d, True, hurst=H, steps=steps,T=T, grid=grid_Glasserman2004, K_low=K_L_GM, traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004, info_cols=inf_cols)
            information.append(inf_list)   

            label_ = f'Anderson Broadie (2004)-{steps}'
            print(''.join(np.repeat('-', 10)), label_, inner_sim_AB, d, ''.join(np.repeat('-', 10)))
            list_inf=AndersonBroadie2004(d, True, hurst=H, steps=steps,T=T, grid= inner_sim_AB, K_low=K_L_AB, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB, info_cols=inf_cols)
            information.append(inf_list)


            label_ = f'Belomestny et al. (2019)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS,  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9,  mode_desai_BBS_BHS='bhs')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) best-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) original-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012) high test-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=int(traj_est_MM*1.5),traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            for lambda_ in [1/20, 1/2, 1]:
                label_ = f'Belomesty (2013); lambda={lambda_}'
                print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
                if lambda_==1/20:
                    traj_est_MM_belomestny2013_=int(1.1*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.1*basis_f_K_U_MM_belomestny2013)
                elif lambda_==1/2:
                    traj_est_MM_belomestny2013_=int(1.05*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.05*basis_f_K_U_MM_belomestny2013)
                else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                list_inf=BelomestnySAA2013(d, True, hurst=H, steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, lambda_=lambda_, p=100)
                inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                information.append(inf_list) 





            with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
                pic.dump(information, fh)
            repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    info_original=pd.DataFrame.from_records(information)
    info_transformed=pd.DataFrame.from_records(utils.information_format_fbm_no_fu(information), columns=utils.header_fbm_no_fu)
    lower_bounds_cv=info_original.iloc[:,[2,13, 14]]
    upper_bounds = info_original.iloc[:, 9]
    loc_lowest_standard_error = lower_bounds_cv.groupby(2)[14].idxmin()
    lower_bounds_cv_sel = lower_bounds_cv.loc[loc_lowest_standard_error, [2, 13]].rename(columns={13: 'best'})
    lower_bounds_cv=pd.merge(left= lower_bounds_cv, right=lower_bounds_cv_sel, on=2)['best']
    lower_bounds_cv.index= info_original.index
    info_transformed['GAP']= upper_bounds- lower_bounds_cv
    table_ = tabulate(utils.information_format_fbm_no_fu(information), headers=utils.header_fbm_no_fu, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)



********** 600 **********
---------- Schoenmakers et al. (2013) KU100 -49 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
Lowerbound
Value 0.5827776886194472
Std 0.002574428468022544
Stop time-mean 25.1606
Stop time-std 13.578917027509963

Upperbound
up 0.6351839880292232
std 0.0013317372570929486
up2 0.6189888495800687
std2 0.0015817665372064172
CV est 0.5819820579514666
CV std 0.0014793176004687483
time avg training 0:03:09.152959
time avg ub 0:00:00.191085
---------- Belom. et al. (2009) KU100-49 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


t= 48
t= 47
t= 46
t= 45
t= 44
t= 43
t= 42
t= 41
t= 40
t= 39
t= 38
t= 37
t= 36
t= 35
t= 34
t= 33
t= 32
t= 31
t= 30
t= 29
t= 28
t= 27
t= 26
t= 25
t= 24
t= 23
t= 22
t= 21
t= 20
t= 19
t= 18
t= 17
t= 16
t= 15
t= 14
t= 13
t= 12
t= 11
t= 10
t= 9
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
[-0.00353239 -0.00356754 -0.00024205]
Lowerbound
Value 0.5823534293774264
Std 0.0025793869778597414
Stop time-mean 25.25913
Stop time-std 13.548964596717346
Upperbound
up 0.6370341003139492
std 0.0013928350247448382
up2 0.619760609814764
std2 0.0016479979245158172
CV est 0.5814735001510007
CV std 0.0015430322429418586
time avg training 0:01:43.522684
time avg ub 0:00:00.110102
---------- Glasserman (2004)-49 1000 1 ----------


...

48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.7439719857915843   - Norm Jacobian 2.969562585332238 - Max abs Jacobian 0.12927200872324793
Iteration: 10, Objective Value: 0.6122959558149428   - Norm Jacobian 1.0029114230291665 - Max abs Jacobian 0.05428391569965265
Iteration: 20, Objective Value: 0.5811071628695801   - Norm Jacobian 0.6841618062061382 - Max abs Jacobian 0.03796450925954014
Iteration: 30, Objective Value: 0.5659825941777357   - Norm Jacobian 0.7749096196090091 - Max abs Jacobian 0.041212390625997804
Iteration: 40, Objective Value: 0.5550752132795124   - Norm Jacobian 0.48007989451114336 - Max abs Jacobian 0.019482667045149196
Iteration: 50, Objective Value: 0.5492478165883795   - Norm Jacobian 0.4430798093006415 - Max abs Jacobian 0.019049079274277004
Iteration: 60, Objective Value: 0.5454477048457946   - Norm Jacobian 0.31137100681566526 - Max abs 

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.9222718674274688   - Norm Jacobian 5.081650658923338 - Max abs Jacobian 0.17381679228692845
Iteration: 10, Objective Value: 0.7014231633763344   - Norm Jacobian 2.4050768842279027 - Max abs Jacobian 0.12712540584445256
Iteration: 20, Objective Value: 0.6561167580473594   - Norm Jacobian 1.3941748279586488 - Max abs Jacobian 0.07693243413618427
Iteration: 30, Objective Value: 0.6400432905361059   - Norm Jacobian 1.195043213489596 - Max abs Jacobian 0.06654776972432998
Iteration: 40, Objective Value: 0.6284800002763758   - Norm Jacobian 0.7016315999061397 - Max abs Jacobian 0.04377394396465252
Iteration: 50, Objective Value: 0.6235008114397773   - Norm Jacobian 0.8306440125038775 - Max abs Jacobian 0.03895444352784286
Iteration: 60, Objective Value: 0.618858235829824   - Norm Jacobian 0.5624849947096077 - Max abs Jacobia

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 1.169486644901062   - Norm Jacobian 7.828380746275312 - Max abs Jacobian 0.48871167752734934
Iteration: 10, Objective Value: 0.8361640437591571   - Norm Jacobian 3.805516698514923 - Max abs Jacobian 0.24810740862976038
Iteration: 20, Objective Value: 0.7443267763352132   - Norm Jacobian 2.902696743875169 - Max abs Jacobian 0.13256798242686008
Iteration: 30, Objective Value: 0.7053566357332919   - Norm Jacobian 1.6590685818393192 - Max abs Jacobian 0.08656785511696839
Iteration: 40, Objective Value: 0.6896604056390905   - Norm Jacobian 1.6848643583150629 - Max abs Jacobian 0.08155496874647328
Iteration: 50, Objective Value: 0.6793528728023442   - Norm Jacobian 1.0415042907749485 - Max abs Jacobian 0.05353867618993025
Iteration: 60, Objective Value: 0.6739150806173956   - Norm Jacobian 1.0109388321927923 - Max abs Jacobian

Test run

In [2]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
auth = Auth.Token('*******') # Authentication has been removed for security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from fBMABLS import main as AndersonBroadie2004
from fBMHK import main as Glassermann
from fBMBelomestny import main as mainBelomestnyetal2009
from fBMSAA import main as SAA_LP
from fBMSAABelomestny import main as BelomestnySAA2013
from fBMSchoenmakersSZH import main as SZH2013

import numpy as np
import pandas as pd
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 700000
traj_est_MM = 2000
traj_test_ub_desai2=20000
traj_est_MM_belomestny2013=6500
traj_lb_testing = 100000
information=[]
steps=49


T=1
basis_f_K_U_MM=30
basis_f_K_U_MM_desai2=20
basis_f_K_U_MM_belomestny2013=60
basis_f_K_U_MM_BHS=int(basis_f_K_U_MM*1.35)

K_L_basic= 90
K_U_belomestny2009= 100
K_U_SZH2013=90
K_L_AndersenBroadie2004=105
K_L_Glasserman2004= 320
grid_Glasserman2004= 1200
inner_sim_SZH_=1
inner_sim_AB= 600 
fine_grid_belomestny2009_= 1

traj_test_ub_AB=traj_test_ub//7
traj_lb_testing_GM=traj_test_ub//4


i=0
if __name__=='__main__':
    for d,H in  [(1,0.2)]:
        for inner_sim in [500]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))     
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols = [d, H, '', '', '', '']


            label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_, d,  ''.join(np.repeat('-', 10)))
            list_inf=SZH2013(d, True, hurst=H, steps=steps,T=T, mode_kaggle=True, grid=1, K_low=K_L_basic,K_up=K_U_SZH2013, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_, info_cols=inf_cols)
            information.append(inf_list)    

            label_=f'Belom. et al. (2009) KU100-{steps}'
            print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_, d,  ''.join(np.repeat('-', 10)))
            list_inf=mainBelomestnyetal2009(d, True,hurst=H, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_, K_low=K_L_basic,K_up=K_U_belomestny2009, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_, info_cols=inf_cols)
            information.append(inf_list)   


            K_L_AB=K_L_AndersenBroadie2004
            K_L_GM=K_L_Glasserman2004
            label_ = f'Glasserman (2004)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004, d, ''.join(np.repeat('-', 10)))
            list_inf=Glassermann(d, True, hurst=H, steps=steps,T=T, grid=grid_Glasserman2004, K_low=K_L_GM, traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004, info_cols=inf_cols)
            information.append(inf_list)   

            label_ = f'Anderson Broadie (2004)-{steps}'
            print(''.join(np.repeat('-', 10)), label_, inner_sim_AB, d, ''.join(np.repeat('-', 10)))
            list_inf=AndersonBroadie2004(d, True, hurst=H, steps=steps,T=T, grid= inner_sim_AB, K_low=K_L_AB, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB, info_cols=inf_cols)
            information.append(inf_list)


            label_ = f'Belomestny et al. (2019)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS,  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9,  mode_desai_BBS_BHS='bhs')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) best-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) original-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012) high test-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=int(traj_est_MM*1.5),traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            for lambda_ in [1/20, 1/2, 1]:
                label_ = f'Belomesty (2013); lambda={lambda_}'
                print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
                if lambda_==1/20:
                    traj_est_MM_belomestny2013_=int(1.1*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.1*basis_f_K_U_MM_belomestny2013)
                elif lambda_==1/2:
                    traj_est_MM_belomestny2013_=int(1.05*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.05*basis_f_K_U_MM_belomestny2013)
                else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                list_inf=BelomestnySAA2013(d, True, hurst=H, steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, lambda_=lambda_, p=100)
                inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                information.append(inf_list) 





            with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
                pic.dump(information, fh)
            repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    info_original=pd.DataFrame.from_records(information)
    info_transformed=pd.DataFrame.from_records(utils.information_format_fbm_no_fu(information), columns=utils.header_fbm_no_fu)
    lower_bounds_cv=info_original.iloc[:,[2,13, 14]]
    upper_bounds = info_original.iloc[:, 9]
    loc_lowest_standard_error = lower_bounds_cv.groupby(2)[14].idxmin()
    lower_bounds_cv_sel = lower_bounds_cv.loc[loc_lowest_standard_error, [2, 13]].rename(columns={13: 'best'})
    lower_bounds_cv=pd.merge(left= lower_bounds_cv, right=lower_bounds_cv_sel, on=2)['best']
    lower_bounds_cv.index= info_original.index
    info_transformed['GAP']= upper_bounds- lower_bounds_cv
    table_ = tabulate(utils.information_format_fbm_no_fu(information), headers=utils.header_fbm_no_fu, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)



********** 500 **********
---------- Schoenmakers et al. (2013) KU100 -49 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
Lowerbound
Value 0.5810332326534801
Std 0.0025581051348849302
Stop time-mean 24.96432
Stop time-std 13.427560721799026

Upperbound
up 0.6348684855124129
std 0.0013110612049320768
up2 0.6201039595211189
std2 0.0015441697855243062
CV est 0.5810696118960444
CV std 0.0014616058629098457
time avg training 0:05:59.863025
time avg ub 0:00:00.241095
---------- Belom. et al. (2009) KU100-49 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


t= 48
t= 47
t= 46
t= 45
t= 44
t= 43
t= 42
t= 41
t= 40
t= 39
t= 38
t= 37
t= 36
t= 35
t= 34
t= 33
t= 32
t= 31
t= 30
t= 29
t= 28
t= 27
t= 26
t= 25
t= 24
t= 23
t= 22
t= 21
t= 20
t= 19
t= 18
t= 17
t= 16
t= 15
t= 14
t= 13
t= 12
t= 11
t= 10
t= 9
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
[-0.00396508 -0.00400699 -0.00132299]
Lowerbound
Value 0.5809302044398507
Std 0.002557175243884531
Stop time-mean 24.92438
Stop time-std 13.432248568858455
Upperbound
up 0.6343512555271216
std 0.0013257385941920943
up2 0.6189652643173221
std2 0.0015609798604880696
CV est 0.580734118162646
CV std 0.0014845241318475863
time avg training 0:03:41.055996
time avg ub 0:00:00.130756
---------- Glasserman (2004)-49 1250 1 ----------


...

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2418399
Academic license 2418399 - for non-commercial use only - registered to p.___@tilburguniversity.edu
Set parameter BarHomogeneous to value 0
Set parameter OptimalityTol to value 0.0001
Set parameter BarConvTol to value 0.0001
Set parameter Crossover to value 0
Set parameter Method to value 2
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Debian GNU/Linux 11 (bullseye)")

CPU model: AMD EPYC 9B14, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2418399 - for non-commercial use only - registered to p.___@tilburguniversity.edu
Optimize a model with 101920 rows, 3921 columns and 98103666 nonzeros
Model fingerprint: 0x289b4753
Coefficient statistics:
  Matrix range     [1

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0



KeyboardInterrupt



Test run

In [1]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
auth = Auth.Token('*******') # Authentication has been removed for security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from fBMABLS import main as AndersonBroadie2004
from fBMHK import main as Glassermann
from fBMBelomestny import main as mainBelomestnyetal2009
from fBMSAA import main as SAA_LP
from fBMSAABelomestny import main as BelomestnySAA2013
from fBMSchoenmakersSZH import main as SZH2013

import numpy as np
import pandas as pd
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 700000
traj_est_MM = 2000
traj_test_ub_desai2=30000
traj_est_MM_belomestny2013=6500
traj_lb_testing = 100000
information=[]
steps=49


T=1
basis_f_K_U_MM=20
basis_f_K_U_MM_desai2=15
basis_f_K_U_MM_belomestny2013=65
basis_f_K_U_MM_BHS=int(basis_f_K_U_MM*1.35)

K_L_basic= 90
K_U_belomestny2009= 100
K_U_SZH2013=90
K_L_AndersenBroadie2004=95
K_L_Glasserman2004= 250
grid_Glasserman2004= 1200
inner_sim_SZH_=1
inner_sim_AB= 600 
fine_grid_belomestny2009_= 1

traj_test_ub_AB=traj_test_ub//7
traj_lb_testing_GM=traj_test_ub//4


i=0
if __name__=='__main__':
    for d,H in  [(1,0.2)]:
        for inner_sim in [500]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))     
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols = [d, H, '', '', '', '']


            label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_, d,  ''.join(np.repeat('-', 10)))
            list_inf=SZH2013(d, True, hurst=H, steps=steps,T=T, mode_kaggle=True, grid=1, K_low=K_L_basic,K_up=K_U_SZH2013, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_, info_cols=inf_cols)
            information.append(inf_list)    

            label_=f'Belom. et al. (2009) KU100-{steps}'
            print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_, d,  ''.join(np.repeat('-', 10)))
            list_inf=mainBelomestnyetal2009(d, True,hurst=H, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_, K_low=K_L_basic,K_up=K_U_belomestny2009, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_, info_cols=inf_cols)
            information.append(inf_list)   


            K_L_AB=K_L_AndersenBroadie2004
            K_L_GM=K_L_Glasserman2004
            label_ = f'Glasserman (2004)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004, d, ''.join(np.repeat('-', 10)))
            list_inf=Glassermann(d, True, hurst=H, steps=steps,T=T, grid=grid_Glasserman2004, K_low=K_L_GM, traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004, info_cols=inf_cols)
            information.append(inf_list)   

            label_ = f'Anderson Broadie (2004)-{steps}'
            print(''.join(np.repeat('-', 10)), label_, inner_sim_AB, d, ''.join(np.repeat('-', 10)))
            list_inf=AndersonBroadie2004(d, True, hurst=H, steps=steps,T=T, grid= inner_sim_AB, K_low=K_L_AB, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB, info_cols=inf_cols)
            information.append(inf_list)


            label_ = f'Belomestny et al. (2019)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS,  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9,  mode_desai_BBS_BHS='bhs')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) best-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) original-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012) high test-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=int(traj_est_MM*1.5),traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            for lambda_ in [1/20, 1/2, 1]:
                label_ = f'Belomesty (2013); lambda={lambda_}'
                print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
                if lambda_==1/20:
                    traj_est_MM_belomestny2013_=int(1.1*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.1*basis_f_K_U_MM_belomestny2013)
                elif lambda_==1/2:
                    traj_est_MM_belomestny2013_=int(1.05*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.05*basis_f_K_U_MM_belomestny2013)
                else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                list_inf=BelomestnySAA2013(d, True, hurst=H, steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, lambda_=lambda_, p=100)
                inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                information.append(inf_list) 





            with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
                pic.dump(information, fh)
            repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    info_original=pd.DataFrame.from_records(information)
    info_transformed=pd.DataFrame.from_records(utils.information_format_fbm_no_fu(information), columns=utils.header_fbm_no_fu)
    lower_bounds_cv=info_original.iloc[:,[2,13, 14]]
    upper_bounds = info_original.iloc[:, 9]
    loc_lowest_standard_error = lower_bounds_cv.groupby(2)[14].idxmin()
    lower_bounds_cv_sel = lower_bounds_cv.loc[loc_lowest_standard_error, [2, 13]].rename(columns={13: 'best'})
    lower_bounds_cv=pd.merge(left= lower_bounds_cv, right=lower_bounds_cv_sel, on=2)['best']
    lower_bounds_cv.index= info_original.index
    info_transformed['GAP']= upper_bounds- lower_bounds_cv
    table_ = tabulate(utils.information_format_fbm_no_fu(information), headers=utils.header_fbm_no_fu, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)



2024-06-13 06:16:20.813291: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-13 06:16:21.470674: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 06:16:21.470753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 06:16:21.586815: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-13 06:16:21.813671: I tensorflow/core/platform/cpu_feature_guar

********** 500 **********
---------- Schoenmakers et al. (2013) KU100 -49 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
Lowerbound
Value 0.5810332326534801
Std 0.0025581051348849302
Stop time-mean 24.96432
Stop time-std 13.427560721799026

Upperbound
up 0.6348684855124129
std 0.0013110612049320768
up2 0.6201039595211189
std2 0.0015441697855243062
CV est 0.5810696118960444
CV std 0.0014616058629098457
time avg training 0:06:08.355681
time avg ub 0:00:00.220165
---------- Belom. et al. (2009) KU100-49 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


...

48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.736844211882402   - Norm Jacobian 2.585566133218493 - Max abs Jacobian 0.13580310102629162
Iteration: 10, Objective Value: 0.6124260113409664   - Norm Jacobian 1.040063134574066 - Max abs Jacobian 0.06368255165789885
Iteration: 20, Objective Value: 0.5848977452216382   - Norm Jacobian 0.5443226458148379 - Max abs Jacobian 0.03807873160361986
Iteration: 30, Objective Value: 0.57200414080385   - Norm Jacobian 0.6144522967412203 - Max abs Jacobian 0.03463789192671094
Iteration: 40, Objective Value: 0.5640551567430594   - Norm Jacobian 0.4764398622105058 - Max abs Jacobian 0.02839670131930489
Iteration: 50, Objective Value: 0.5597943561248229   - Norm Jacobian 0.27069423762408235 - Max abs Jacobian 0.014889293324393055
Iteration: 60, Objective Value: 0.5573304695416343   - Norm Jacobian 0.21955512010771974 - Max abs Jacobi

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.8805608267403581   - Norm Jacobian 4.407369321356996 - Max abs Jacobian 0.20047417780037702
Iteration: 10, Objective Value: 0.6931579478656985   - Norm Jacobian 1.6598067956424232 - Max abs Jacobian 0.08587291390970538
Iteration: 20, Objective Value: 0.6571847608918547   - Norm Jacobian 0.8689669799398168 - Max abs Jacobian 0.04774878493915183
Iteration: 30, Objective Value: 0.6448603524554284   - Norm Jacobian 0.7741183286166688 - Max abs Jacobian 0.03981276758783094
Iteration: 40, Objective Value: 0.6366075605712067   - Norm Jacobian 0.5369868427641381 - Max abs Jacobian 0.034076152849050925
Iteration: 50, Objective Value: 0.6320262419433303   - Norm Jacobian 0.3414934792016301 - Max abs Jacobian 0.024579170710998327
Iteration: 60, Objective Value: 0.6295155945258603   - Norm Jacobian 0.34485855181131086 - Max abs Ja

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 1.1136098692122027   - Norm Jacobian 7.172891050499559 - Max abs Jacobian 0.5214162829706706
Iteration: 10, Objective Value: 0.8003202253002732   - Norm Jacobian 3.138480781949184 - Max abs Jacobian 0.23642942243104326
Iteration: 20, Objective Value: 0.7378752084905144   - Norm Jacobian 2.5463745916409413 - Max abs Jacobian 0.10250013482109244
Iteration: 30, Objective Value: 0.7062803028266742   - Norm Jacobian 1.2941677234154447 - Max abs Jacobian 0.08927507737353409
Iteration: 40, Objective Value: 0.6943985119004777   - Norm Jacobian 1.2206904494635895 - Max abs Jacobian 0.08229724846873752
Iteration: 50, Objective Value: 0.6882649036122553   - Norm Jacobian 1.0408265756807813 - Max abs Jacobian 0.06883298005176325
Iteration: 60, Objective Value: 0.6834024955022983   - Norm Jacobian 0.8152230704603434 - Max abs Jacobia

Test run

In [1]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
auth = Auth.Token('*******') # Authentication has been removed for security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from fBMABLS import main as AndersonBroadie2004
from fBMHK import main as Glassermann
from fBMBelomestny import main as mainBelomestnyetal2009
from fBMSAA import main as SAA_LP
from fBMSAABelomestny import main as BelomestnySAA2013
from fBMSchoenmakersSZH import main as SZH2013

import numpy as np
import pandas as pd
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 1000000
traj_est_MM = 2000
traj_test_ub_desai2=30000
traj_est_MM_belomestny2013=6500
traj_lb_testing = 100000
information=[]
steps=100


T=1
basis_f_K_U_MM=20
basis_f_K_U_MM_desai2=15
basis_f_K_U_MM_belomestny2013=65
basis_f_K_U_MM_BHS=int(basis_f_K_U_MM*1.35)

K_L_basic= 90
K_U_belomestny2009= 100
K_U_SZH2013=90
K_L_AndersenBroadie2004=50
K_L_Glasserman2004= 250
grid_Glasserman2004= 1024
inner_sim_SZH_=1
inner_sim_AB= 1024 
fine_grid_belomestny2009_= 1

traj_test_ub_AB=traj_test_ub//7
traj_lb_testing_GM=traj_test_ub//4


i=0
if __name__=='__main__':
    for d,H in  [(1,0.2), (1, 0.45)]:
        for inner_sim in [500]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))     
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols = [d, H, '', '', '', '']


#             label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_, d,  ''.join(np.repeat('-', 10)))
#             list_inf=SZH2013(d, True, hurst=H, steps=steps,T=T, mode_kaggle=True, grid=1, K_low=K_L_basic,K_up=K_U_SZH2013, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9)
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_, info_cols=inf_cols)
#             information.append(inf_list)    

#             label_=f'Belom. et al. (2009) KU100-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_, d,  ''.join(np.repeat('-', 10)))
#             list_inf=mainBelomestnyetal2009(d, True,hurst=H, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_, K_low=K_L_basic,K_up=K_U_belomestny2009, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9) 
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_, info_cols=inf_cols)
#             information.append(inf_list)   


            K_L_AB=K_L_AndersenBroadie2004
            K_L_GM=K_L_Glasserman2004
            label_ = f'Glasserman (2004)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004, d, ''.join(np.repeat('-', 10)))
            list_inf=Glassermann(d, True, hurst=H, steps=steps,T=T, grid=grid_Glasserman2004, K_low=K_L_GM, traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004, info_cols=inf_cols)
            information.append(inf_list)   

            label_ = f'Anderson Broadie (2004)-{steps}'
            print(''.join(np.repeat('-', 10)), label_, inner_sim_AB, d, ''.join(np.repeat('-', 10)))
            list_inf=AndersonBroadie2004(d, True, hurst=H, steps=steps,T=T, grid= inner_sim_AB, K_low=K_L_AB, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB, info_cols=inf_cols)
            information.append(inf_list)


#             label_ = f'Belomestny et al. (2019)-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
#             list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS,  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9,  mode_desai_BBS_BHS='bhs')
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
#             information.append(inf_list)  

#             label_ = f'Belomestny et al. (2023) best-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
#             list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
#             information.append(inf_list)  

#             label_ = f'Belomestny et al. (2023) original-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
#             list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
#             information.append(inf_list)  

#             label_ =f'Desai et al. (2012)-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
#             list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
#             information.append(inf_list)  

#             label_ =f'Desai et al. (2012) high test-{steps}'
#             print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
#             list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=int(traj_est_MM*1.5),traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
#             inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
#             information.append(inf_list)  

#             for lambda_ in [1/20, 1/2, 1]:
#                 label_ = f'Belomesty (2013); lambda={lambda_}'
#                 print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
#                 if lambda_==1/20:
#                     traj_est_MM_belomestny2013_=int(1.1*traj_est_MM_belomestny2013)
#                     basis_f_K_U_MM_belomestny2013_=int(1.1*basis_f_K_U_MM_belomestny2013)
#                 elif lambda_==1/2:
#                     traj_est_MM_belomestny2013_=int(1.05*traj_est_MM_belomestny2013)
#                     basis_f_K_U_MM_belomestny2013_=int(1.05*basis_f_K_U_MM_belomestny2013)
#                 else:
#                     traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
#                     basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

#                 list_inf=BelomestnySAA2013(d, True, hurst=H, steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, lambda_=lambda_, p=100)
#                 inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
#                 information.append(inf_list) 





            with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
                pic.dump(information, fh)
            repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    info_original=pd.DataFrame.from_records(information)
    info_transformed=pd.DataFrame.from_records(utils.information_format_fbm_no_fu(information), columns=utils.header_fbm_no_fu)
    lower_bounds_cv=info_original.iloc[:,[2,13, 14]]
    upper_bounds = info_original.iloc[:, 9]
    loc_lowest_standard_error = lower_bounds_cv.groupby(2)[14].idxmin()
    lower_bounds_cv_sel = lower_bounds_cv.loc[loc_lowest_standard_error, [2, 13]].rename(columns={13: 'best'})
    lower_bounds_cv=pd.merge(left= lower_bounds_cv, right=lower_bounds_cv_sel, on=2)['best']
    lower_bounds_cv.index= info_original.index
    info_transformed['GAP']= upper_bounds- lower_bounds_cv
    table_ = tabulate(utils.information_format_fbm_no_fu(information), headers=utils.header_fbm_no_fu, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)



2024-06-13 20:52:26.405221: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-13 20:52:27.034545: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 20:52:27.034622: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 20:52:27.150053: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-13 20:52:27.364200: I tensorflow/core/platform/cpu_feature_guar

********** 500 **********
---------- Glasserman (2004)-100 1024 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
0.6564088959309581
[-0.01212567 -0.0016239  -0.00349797]
Lowerbound
Value 0.6542454568524043
Std 0.002548526337873634
Stop time-mean 50.58207
Stop time-std 27.635963245653297
Upperbound
up 0.6677369987826628
std 0.001188399800526311
up2 0.6642424342044653
std2 0.0012432823666464247
CV est 0.6547058595697944
CV std 0.0011377328284639663
time avg training 0:13:24.408167
time avg ub 0:05:56.984124
---------- Anderson Broadie (2004)-100 1024 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


...

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
0.058136833375047436
[-0.01414428 -0.01075856 -0.0114112 ]
Lowerbound
Value 0.056354091694095956
Std 0.0017360852100933367
Stop time-mean 28.91587
Stop time-std 20.118738333779785
Upperbound
up 0.10379334140565606
std 0.0011369104761310472
up2 0.10229170649636112
std2 0.0011710198409094264
CV est 0.055886849986188335
CV std 0.0005656750293985025
time avg training 0:13:21.737075
time avg ub 0:05:55.556654
---------- Anderson Broadie (2004)-100 1024 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37


ValueError: need at least one array to concatenate

Test run

In [1]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
auth = Auth.Token('*******') # Authentication has been removed for security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from fBMABLS import main as AndersonBroadie2004
from fBMHK import main as Glassermann
from fBMBelomestny import main as mainBelomestnyetal2009
from fBMSAA import main as SAA_LP
from fBMSAABelomestny import main as BelomestnySAA2013
from fBMSchoenmakersSZH import main as SZH2013

import numpy as np
import pandas as pd
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 700000
traj_est_MM = 2000
traj_test_ub_desai2=30000
traj_est_MM_belomestny2013=6500
traj_lb_testing = 100000
information=[]
steps=49


T=1
basis_f_K_U_MM=20
basis_f_K_U_MM_desai2=15
basis_f_K_U_MM_belomestny2013=65
basis_f_K_U_MM_BHS=int(basis_f_K_U_MM*1.35)

K_L_basic= 90
K_U_belomestny2009= 100
K_U_SZH2013=90
K_L_AndersenBroadie2004=95
K_L_Glasserman2004= 250
grid_Glasserman2004= 1200
inner_sim_SZH_=1
inner_sim_AB= 600 
fine_grid_belomestny2009_= 1

traj_test_ub_AB=traj_test_ub//7
traj_lb_testing_GM=traj_test_ub//4


i=0
if __name__=='__main__':
    for d,H in  [(1,0.45)]:
        for inner_sim in [500]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))     
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols = [d, H, '', '', '', '']


            label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_, d,  ''.join(np.repeat('-', 10)))
            list_inf=SZH2013(d, True, hurst=H, steps=steps,T=T, mode_kaggle=True, grid=1, K_low=K_L_basic,K_up=K_U_SZH2013, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_, info_cols=inf_cols)
            information.append(inf_list)    

            label_=f'Belom. et al. (2009) KU100-{steps}'
            print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_, d,  ''.join(np.repeat('-', 10)))
            list_inf=mainBelomestnyetal2009(d, True,hurst=H, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_, K_low=K_L_basic,K_up=K_U_belomestny2009, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_, info_cols=inf_cols)
            information.append(inf_list)   


            K_L_AB=K_L_AndersenBroadie2004
            K_L_GM=K_L_Glasserman2004
            label_ = f'Glasserman (2004)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004, d, ''.join(np.repeat('-', 10)))
            list_inf=Glassermann(d, True, hurst=H, steps=steps,T=T, grid=grid_Glasserman2004, K_low=K_L_GM, traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004, info_cols=inf_cols)
            information.append(inf_list)   

            label_ = f'Anderson Broadie (2004)-{steps}'
            print(''.join(np.repeat('-', 10)), label_, inner_sim_AB, d, ''.join(np.repeat('-', 10)))
            list_inf=AndersonBroadie2004(d, True, hurst=H, steps=steps,T=T, grid= inner_sim_AB, K_low=K_L_AB, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB, info_cols=inf_cols)
            information.append(inf_list)


            label_ = f'Belomestny et al. (2019)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS,  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9,  mode_desai_BBS_BHS='bhs')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) best-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) original-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012) high test-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=int(traj_est_MM*1.5),traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            for lambda_ in [1/20, 1/2, 1]:
                label_ = f'Belomesty (2013); lambda={lambda_}'
                print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
                if lambda_==1/20:
                    traj_est_MM_belomestny2013_=int(1.1*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.1*basis_f_K_U_MM_belomestny2013)
                elif lambda_==1/2:
                    traj_est_MM_belomestny2013_=int(1.05*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.05*basis_f_K_U_MM_belomestny2013)
                else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                list_inf=BelomestnySAA2013(d, True, hurst=H, steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, lambda_=lambda_, p=100)
                inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                information.append(inf_list) 





            with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
                pic.dump(information, fh)
            repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    info_original=pd.DataFrame.from_records(information)
    info_transformed=pd.DataFrame.from_records(utils.information_format_fbm_no_fu(information), columns=utils.header_fbm_no_fu)
    lower_bounds_cv=info_original.iloc[:,[2,13, 14]]
    upper_bounds = info_original.iloc[:, 9]
    loc_lowest_standard_error = lower_bounds_cv.groupby(2)[14].idxmin()
    lower_bounds_cv_sel = lower_bounds_cv.loc[loc_lowest_standard_error, [2, 13]].rename(columns={13: 'best'})
    lower_bounds_cv=pd.merge(left= lower_bounds_cv, right=lower_bounds_cv_sel, on=2)['best']
    lower_bounds_cv.index= info_original.index
    info_transformed['GAP']= upper_bounds- lower_bounds_cv
    table_ = tabulate(utils.information_format_fbm_no_fu(information), headers=utils.header_fbm_no_fu, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)



2024-06-14 06:10:47.822115: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-14 06:10:48.475755: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 06:10:48.475834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 06:10:48.592388: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-14 06:10:48.809656: I tensorflow/core/platform/cpu_feature_guar

********** 500 **********
---------- Schoenmakers et al. (2013) KU100 -49 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
Lowerbound
Value 0.046133100612136504
Std 0.0016780628885104353
Stop time-mean 12.97347
Stop time-std 8.4094010582859

Upperbound
up 0.0902213924980675
std 0.0004672286666147503
up2 0.08970538048869178
std2 0.00047365245194081986
CV est 0.04698159594960505
CV std 0.0001837683159039291
time avg training 0:06:47.027122
time avg ub 0:00:00.242806
---------- Belom. et al. (2009) KU100-49 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


...

48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.2490466186643049   - Norm Jacobian 4.853085407859707 - Max abs Jacobian 0.16064350019712964
Iteration: 10, Objective Value: 0.1291818465053571   - Norm Jacobian 3.0238201542042518 - Max abs Jacobian 0.12862820076260742
Iteration: 20, Objective Value: 0.09637517061396059   - Norm Jacobian 1.9036828020399543 - Max abs Jacobian 0.09960932038356399
Iteration: 30, Objective Value: 0.08844946933129508   - Norm Jacobian 1.0655394512800038 - Max abs Jacobian 0.049130438975149696
Iteration: 40, Objective Value: 0.08509226437014004   - Norm Jacobian 0.6129821172150891 - Max abs Jacobian 0.026524148141746678
Iteration: 50, Objective Value: 0.0838132207111018   - Norm Jacobian 0.5473088761192004 - Max abs Jacobian 0.03120782870509409
Iteration: 60, Objective Value: 0.082995956916466   - Norm Jacobian 0.5860812171555029 - Max abs J

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.3572313837832786   - Norm Jacobian 6.781980287262975 - Max abs Jacobian 0.3010532102271857
Iteration: 10, Objective Value: 0.1694915564443047   - Norm Jacobian 3.4179416483935405 - Max abs Jacobian 0.18185741381537837
Iteration: 20, Objective Value: 0.12046899855524323   - Norm Jacobian 2.846437671980136 - Max abs Jacobian 0.18103939575030734
Iteration: 30, Objective Value: 0.10822457885707625   - Norm Jacobian 1.8293043270974103 - Max abs Jacobian 0.10525062160117543
Iteration: 40, Objective Value: 0.10316690741798212   - Norm Jacobian 1.231770073685853 - Max abs Jacobian 0.048126955344302315
Iteration: 50, Objective Value: 0.101187346557263   - Norm Jacobian 1.1426394204199617 - Max abs Jacobian 0.08900081010112927
Iteration: 60, Objective Value: 0.09963437830638222   - Norm Jacobian 0.7737082332205222 - Max abs Jaco

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.4549744828977618   - Norm Jacobian 8.834562364343565 - Max abs Jacobian 0.436534112316941
Iteration: 10, Objective Value: 0.25974537871832654   - Norm Jacobian 5.173132254279649 - Max abs Jacobian 0.27713941043507045
Iteration: 20, Objective Value: 0.1641598798378315   - Norm Jacobian 4.778928630970034 - Max abs Jacobian 0.2674536088438793
Iteration: 30, Objective Value: 0.1397377155372919   - Norm Jacobian 3.8714509850032544 - Max abs Jacobian 0.22161115604604584
Iteration: 40, Objective Value: 0.1247921878243528   - Norm Jacobian 2.309979462869801 - Max abs Jacobian 0.119057281466502
Iteration: 50, Objective Value: 0.120772396267214   - Norm Jacobian 2.0183519292274377 - Max abs Jacobian 0.12926534146118032
Iteration: 60, Objective Value: 0.11719012404658997   - Norm Jacobian 1.6499557058105787 - Max abs Jacobian 0.0

## Final Run N_T=49 

In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
auth = Auth.Token('*******') # Authentication has been removed for security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from fBMABLS import main as AndersonBroadie2004
from fBMHK import main as Glassermann
from fBMBelomestny import main as mainBelomestnyetal2009
from fBMSAA import main as SAA_LP
from fBMSAABelomestny import main as BelomestnySAA2013
from fBMSchoenmakersSZH import main as SZH2013

import numpy as np
import pandas as pd
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 700000
traj_est_MM = 2000
traj_test_ub_desai2=30000
traj_est_MM_belomestny2013=6500
traj_lb_testing = 100000
information=[]
steps=49


T=1
basis_f_K_U_MM=20
basis_f_K_U_MM_desai2=15
basis_f_K_U_MM_belomestny2013=65
basis_f_K_U_MM_BHS=int(basis_f_K_U_MM*1.35)

K_L_basic= 90
K_U_belomestny2009= 100
K_U_SZH2013=90
K_L_AndersenBroadie2004=100
K_L_Glasserman2004= 225
grid_Glasserman2004= 1200
inner_sim_SZH_=1
inner_sim_AB= 650 
fine_grid_belomestny2009_= 1

traj_test_ub_AB=traj_test_ub//7
traj_lb_testing_GM=traj_test_ub//4


i=0
if __name__=='__main__':
    for d,H in  [(1, 0.2), (1,0.45)]:
        for inner_sim in [500]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))     
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols = [d, H, '', '', '', '']


            label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_, d,  ''.join(np.repeat('-', 10)))
            list_inf=SZH2013(d, True, hurst=H, steps=steps,T=T, mode_kaggle=True, grid=1, K_low=K_L_basic,K_up=K_U_SZH2013, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_, info_cols=inf_cols)
            information.append(inf_list)    

            label_=f'Belom. et al. (2009) KU100-{steps}'
            print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_, d,  ''.join(np.repeat('-', 10)))
            list_inf=mainBelomestnyetal2009(d, True,hurst=H, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_, K_low=K_L_basic,K_up=K_U_belomestny2009, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_, info_cols=inf_cols)
            information.append(inf_list)   


            K_L_AB=K_L_AndersenBroadie2004
            K_L_GM=K_L_Glasserman2004
            label_ = f'Glasserman (2004)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004, d, ''.join(np.repeat('-', 10)))
            list_inf=Glassermann(d, True, hurst=H, steps=steps,T=T, grid=grid_Glasserman2004, K_low=K_L_GM, traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004, info_cols=inf_cols)
            information.append(inf_list)   

            label_ = f'Anderson Broadie (2004)-{steps}'
            print(''.join(np.repeat('-', 10)), label_, inner_sim_AB, d, ''.join(np.repeat('-', 10)))
            list_inf=AndersonBroadie2004(d, True, hurst=H, steps=steps,T=T, grid= inner_sim_AB, K_low=K_L_AB, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB, info_cols=inf_cols)
            information.append(inf_list)


            label_ = f'Belomestny et al. (2019)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS,  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9,  mode_desai_BBS_BHS='bhs')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) best-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) original-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012) high test-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=int(traj_est_MM*1.5),traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            for lambda_ in [1/20, 1/2, 1]:
                label_ = f'Belomesty (2013); lambda={lambda_}'
                print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
                if lambda_==1/20:
                    traj_est_MM_belomestny2013_=int(1.1*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.1*basis_f_K_U_MM_belomestny2013)
                elif lambda_==1/2:
                    traj_est_MM_belomestny2013_=int(1.05*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.05*basis_f_K_U_MM_belomestny2013)
                else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                list_inf=BelomestnySAA2013(d, True, hurst=H, steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, lambda_=lambda_, p=100)
                inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                information.append(inf_list) 





            with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
                pic.dump(information, fh)
            repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    info_original=pd.DataFrame.from_records(information)
    info_transformed=pd.DataFrame.from_records(utils.information_format_fbm_no_fu(information), columns=utils.header_fbm_no_fu)
    lower_bounds_cv=info_original.iloc[:,[2,13, 14]]
    upper_bounds = info_original.iloc[:, 9]
    loc_lowest_standard_error = lower_bounds_cv.groupby(2)[14].idxmin()
    lower_bounds_cv_sel = lower_bounds_cv.loc[loc_lowest_standard_error, [2, 13]].rename(columns={13: 'best'})
    lower_bounds_cv=pd.merge(left= lower_bounds_cv, right=lower_bounds_cv_sel, on=2)['best']
    lower_bounds_cv.index= info_original.index
    info_transformed['GAP']= upper_bounds- lower_bounds_cv
    table_ = tabulate(utils.information_format_fbm_no_fu(information), headers=utils.header_fbm_no_fu, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)



********** 500 **********
---------- Schoenmakers et al. (2013) KU100 -49 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
Lowerbound
Value 0.5810332326534801
Std 0.0025581051348849302
Stop time-mean 24.96432
Stop time-std 13.427560721799026

Upperbound
up 0.6348684855124129
std 0.0013110612049320768
up2 0.6201039595211189
std2 0.0015441697855243062
CV est 0.5810696118960444
CV std 0.0014616058629098457
time avg training 0:05:57.393324
time avg ub 0:00:00.239095
---------- Belom. et al. (2009) KU100-49 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


t= 48
t= 47
t= 46
t= 45
t= 44
t= 43
t= 42
t= 41
t= 40
t= 39
t= 38
t= 37
t= 36
t= 35
t= 34
t= 33
t= 32
t= 31
t= 30
t= 29
t= 28
t= 27
t= 26
t= 25
t= 24
t= 23
t= 22
t= 21
t= 20
t= 19
t= 18
t= 17
t= 16
t= 15
t= 14
t= 13
t= 12
t= 11
t= 10
t= 9
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
[-0.00396508 -0.00400699 -0.00132299]
Lowerbound
Value 0.5809302044398507
Std 0.002557175243884531
Stop time-mean 24.92438
Stop time-std 13.432248568858455
Upperbound
up 0.6343512555271216
std 0.0013257385941920943
up2 0.6189652643173221
std2 0.0015609798604880696
CV est 0.580734118162646
CV std 0.0014845241318475863
time avg training 0:03:42.231468
time avg ub 0:00:00.165870
---------- Glasserman (2004)-49 1200 1 ----------


...

48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.2490466186643049   - Norm Jacobian 4.853085407859707 - Max abs Jacobian 0.16064350019712964
Iteration: 10, Objective Value: 0.1291818465053571   - Norm Jacobian 3.0238201542042518 - Max abs Jacobian 0.12862820076260742
Iteration: 20, Objective Value: 0.09637517061396059   - Norm Jacobian 1.9036828020399543 - Max abs Jacobian 0.09960932038356399
Iteration: 30, Objective Value: 0.08844946933129508   - Norm Jacobian 1.0655394512800038 - Max abs Jacobian 0.049130438975149696
Iteration: 40, Objective Value: 0.08509226437014004   - Norm Jacobian 0.6129821172150891 - Max abs Jacobian 0.026524148141746678
Iteration: 50, Objective Value: 0.0838132207111018   - Norm Jacobian 0.5473088761192004 - Max abs Jacobian 0.03120782870509409
Iteration: 60, Objective Value: 0.082995956916466   - Norm Jacobian 0.5860812171555029 - Max abs J

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.3572313837832786   - Norm Jacobian 6.781980287262975 - Max abs Jacobian 0.3010532102271857
Iteration: 10, Objective Value: 0.1694915564443047   - Norm Jacobian 3.4179416483935405 - Max abs Jacobian 0.18185741381537837
Iteration: 20, Objective Value: 0.12046899855524323   - Norm Jacobian 2.846437671980136 - Max abs Jacobian 0.18103939575030734
Iteration: 30, Objective Value: 0.10822457885707625   - Norm Jacobian 1.8293043270974103 - Max abs Jacobian 0.10525062160117543
Iteration: 40, Objective Value: 0.10316690741798212   - Norm Jacobian 1.231770073685853 - Max abs Jacobian 0.048126955344302315
Iteration: 50, Objective Value: 0.101187346557263   - Norm Jacobian 1.1426394204199617 - Max abs Jacobian 0.08900081010112927
Iteration: 60, Objective Value: 0.09963437830638222   - Norm Jacobian 0.7737082332205222 - Max abs Jaco

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.4549744828977618   - Norm Jacobian 8.834562364343565 - Max abs Jacobian 0.436534112316941
Iteration: 10, Objective Value: 0.25974537871832654   - Norm Jacobian 5.173132254279649 - Max abs Jacobian 0.27713941043507045
Iteration: 20, Objective Value: 0.1641598798378315   - Norm Jacobian 4.778928630970034 - Max abs Jacobian 0.2674536088438793
Iteration: 30, Objective Value: 0.1397377155372919   - Norm Jacobian 3.8714509850032544 - Max abs Jacobian 0.22161115604604584
Iteration: 40, Objective Value: 0.1247921878243528   - Norm Jacobian 2.309979462869801 - Max abs Jacobian 0.119057281466502
Iteration: 50, Objective Value: 0.120772396267214   - Norm Jacobian 2.0183519292274377 - Max abs Jacobian 0.12926534146118032
Iteration: 60, Objective Value: 0.11719012404658997   - Norm Jacobian 1.6499557058105787 - Max abs Jacobian 0.0

Test run (higher number of basis functions and improved LSMC primal stopping rules)

In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
auth = Auth.Token('*******') # Authentication has been removed for security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from fBMABLS import main as AndersonBroadie2004
from fBMHK import main as Glassermann
from fBMBelomestny import main as mainBelomestnyetal2009
from fBMSAA import main as SAA_LP
from fBMSAABelomestny import main as BelomestnySAA2013
from fBMSchoenmakersSZH import main as SZH2013

import numpy as np
import pandas as pd
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 700000
traj_est_MM = 2000
traj_test_ub_desai2=30000
traj_est_MM_belomestny2013=6500
traj_lb_testing = 100000
information=[]
steps=49


T=1
basis_f_K_U_MM=20
basis_f_K_U_MM_desai2=15
basis_f_K_U_MM_belomestny2013=65
basis_f_K_U_MM_BHS=int(basis_f_K_U_MM*1.35)

K_L_basic=200
K_U_belomestny2009= 110
K_U_SZH2013=75
K_L_AndersenBroadie2004=200
K_L_Glasserman2004= 225
grid_Glasserman2004= 1200
inner_sim_SZH_=1
inner_sim_AB= 400 
fine_grid_belomestny2009_= 1

traj_test_ub_AB=traj_test_ub//7
traj_lb_testing_GM=traj_test_ub//4


i=0
if __name__=='__main__':
    for d,H in  [(1,0.2), (1,0.45)]:
        for inner_sim in [500]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))     
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols = [d, H, '', '', '', '']


            label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_, d,  ''.join(np.repeat('-', 10)))
            list_inf=SZH2013(d, True, hurst=H, steps=steps,T=T, mode_kaggle=True, grid=1, K_low=K_L_basic,K_up=K_U_SZH2013, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_, info_cols=inf_cols)
            information.append(inf_list)    

            label_=f'Belom. et al. (2009) KU100-{steps}'
            print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_, d,  ''.join(np.repeat('-', 10)))
            list_inf=mainBelomestnyetal2009(d, True,hurst=H, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_, K_low=K_L_basic,K_up=K_U_belomestny2009, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_, info_cols=inf_cols)
            information.append(inf_list)   


            K_L_AB=K_L_AndersenBroadie2004
            K_L_GM=K_L_Glasserman2004
            label_ = f'Glasserman (2004)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004, d, ''.join(np.repeat('-', 10)))
            list_inf=Glassermann(d, True, hurst=H, steps=steps,T=T, grid=grid_Glasserman2004, K_low=K_L_GM, traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004, info_cols=inf_cols)
            information.append(inf_list)   

            label_ = f'Anderson Broadie (2004)-{steps}'
            print(''.join(np.repeat('-', 10)), label_, inner_sim_AB, d, ''.join(np.repeat('-', 10)))
            list_inf=AndersonBroadie2004(d, True, hurst=H, steps=steps,T=T, grid= inner_sim_AB, K_low=K_L_AB, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB, info_cols=inf_cols)
            information.append(inf_list)


            label_ = f'Belomestny et al. (2019)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS,  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9,  mode_desai_BBS_BHS='bhs')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) best-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) original-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012) high test-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=int(traj_est_MM*1.5),traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            for lambda_ in [1/20, 1/2, 1]:
                label_ = f'Belomesty (2013); lambda={lambda_}'
                print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
                if lambda_==1/20:
                    traj_est_MM_belomestny2013_=int(1.1*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.1*basis_f_K_U_MM_belomestny2013)
                elif lambda_==1/2:
                    traj_est_MM_belomestny2013_=int(1.05*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.05*basis_f_K_U_MM_belomestny2013)
                else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                list_inf=BelomestnySAA2013(d, True, hurst=H, steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, lambda_=lambda_, p=100)
                inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                information.append(inf_list) 





            with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
                pic.dump(information, fh)
            repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    info_original=pd.DataFrame.from_records(information)
    info_transformed=pd.DataFrame.from_records(utils.information_format_fbm_no_fu(information), columns=utils.header_fbm_no_fu)
    lower_bounds_cv=info_original.iloc[:,[2,13, 14]]
    upper_bounds = info_original.iloc[:, 9]
    loc_lowest_standard_error = lower_bounds_cv.groupby(2)[14].idxmin()
    lower_bounds_cv_sel = lower_bounds_cv.loc[loc_lowest_standard_error, [2, 13]].rename(columns={13: 'best'})
    lower_bounds_cv=pd.merge(left= lower_bounds_cv, right=lower_bounds_cv_sel, on=2)['best']
    lower_bounds_cv.index= info_original.index
    info_transformed['GAP']= upper_bounds- lower_bounds_cv
    table_ = tabulate(utils.information_format_fbm_no_fu(information), headers=utils.header_fbm_no_fu, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)



********** 500 **********
---------- Schoenmakers et al. (2013) KU100 -49 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
Lowerbound
Value 0.5859255660388076
Std 0.0025825279435969114
Stop time-mean 25.43839
Stop time-std 13.706915926199446

Upperbound
up 0.6373973294320792
std 0.0013285695542543463
up2 0.6206365803387507
std2 0.001594869477626332
CV est 0.5857335054302835
CV std 0.0014919576788036553
time avg training 0:08:19.332410
time avg ub 0:00:00.229788
---------- Belom. et al. (2009) KU100-49 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


t= 48
t= 47
t= 46
t= 45
t= 44
t= 43
t= 42
t= 41
t= 40
t= 39
t= 38
t= 37
t= 36
t= 35
t= 34
t= 33
t= 32
t= 31
t= 30
t= 29
t= 28
t= 27
t= 26
t= 25
t= 24
t= 23
t= 22
t= 21
t= 20
t= 19
t= 18
t= 17
t= 16
t= 15
t= 14
t= 13
t= 12
t= 11
t= 10
t= 9
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
[-0.00574412 -0.00538482 -0.00262184]
Lowerbound
Value 0.5850778762878759
Std 0.002582906654153012
Stop time-mean 25.44513
Stop time-std 13.708763229522201
Upperbound
up 0.6342028982122626
std 0.0013266994676337655
up2 0.6177704355596199
std2 0.001573988001292028
CV est 0.5847737067389689
CV std 0.0014733946882024252
time avg training 0:06:02.138838
time avg ub 0:00:00.179232
---------- Glasserman (2004)-49 1200 1 ----------


...

48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.24727619771078688   - Norm Jacobian 4.850604886255864 - Max abs Jacobian 0.15942071682903336
Iteration: 10, Objective Value: 0.11622681154553358   - Norm Jacobian 2.4145405779932565 - Max abs Jacobian 0.16001715337702277
Iteration: 20, Objective Value: 0.09336059762527725   - Norm Jacobian 1.6239484676840767 - Max abs Jacobian 0.0779123992024902
Iteration: 30, Objective Value: 0.0873191009416969   - Norm Jacobian 1.0537869005309142 - Max abs Jacobian 0.052971953788695354
Iteration: 40, Objective Value: 0.08462020502101883   - Norm Jacobian 0.5759843290568333 - Max abs Jacobian 0.03737198917544012
Iteration: 50, Objective Value: 0.08339734912464977   - Norm Jacobian 0.44968301598739047 - Max abs Jacobian 0.02981855448172784
Iteration: 60, Objective Value: 0.08258621783233491   - Norm Jacobian 0.41288546699447226 - Max a

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.35621462098449197   - Norm Jacobian 6.656005740088506 - Max abs Jacobian 0.29495679436616873
Iteration: 10, Objective Value: 0.18072765280792188   - Norm Jacobian 3.676440314497514 - Max abs Jacobian 0.1686046595096042
Iteration: 20, Objective Value: 0.1340984065639113   - Norm Jacobian 4.054589818442622 - Max abs Jacobian 0.2366064548527852
Iteration: 30, Objective Value: 0.1143363590921129   - Norm Jacobian 2.7152077570155684 - Max abs Jacobian 0.10808285711031529
Iteration: 40, Objective Value: 0.10516391872718318   - Norm Jacobian 1.7496198151648812 - Max abs Jacobian 0.09021332546422121
Iteration: 50, Objective Value: 0.10175441163020497   - Norm Jacobian 1.3203931438714256 - Max abs Jacobian 0.07214921116172036
Iteration: 60, Objective Value: 0.09984103926345338   - Norm Jacobian 0.9838455883787716 - Max abs Jaco

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.4598623831305267   - Norm Jacobian 8.822615263229842 - Max abs Jacobian 0.41763310002872
Iteration: 10, Objective Value: 0.25989381268122597   - Norm Jacobian 5.011465845394628 - Max abs Jacobian 0.2837080269460883
Iteration: 20, Objective Value: 0.1579279377267591   - Norm Jacobian 4.119344486800211 - Max abs Jacobian 0.2838048156424613
Iteration: 30, Objective Value: 0.13790564343240727   - Norm Jacobian 3.416380756585304 - Max abs Jacobian 0.18045536159204362
Iteration: 40, Objective Value: 0.1264747429266479   - Norm Jacobian 2.401441316833915 - Max abs Jacobian 0.13697801571838097
Iteration: 50, Objective Value: 0.12093503028382609   - Norm Jacobian 1.9791195819732783 - Max abs Jacobian 0.08861898950828451
Iteration: 60, Objective Value: 0.11789129750617997   - Norm Jacobian 1.7078726782794629 - Max abs Jacobian 0